# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We transformed "RAD" and "TAX" to dummy variables and dropped the first variable.
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0, 6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX", drop_first=True).rename(columns=lambda col: col.replace(" ", "").replace(",", "_").replace("(", "").replace("]", ""))
print(tax_dummy.columns.tolist())
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD", drop_first=True).rename(columns=lambda col: col.replace(" ", "").replace(",", "_").replace("(", "").replace("]", ""))
print(rad_dummy.columns.tolist())
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

['TAX_270_360', 'TAX_360_712']
['RAD_6_24']


In [2]:
age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [3]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,RAD_6_24,TAX_270_360,TAX_360_712
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,0,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,0,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,0,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,0,0,0


## Run an linear model in Statsmodels

In [4]:
# Your code here
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression

def lin_reg_summary(df, target, pred, t = 0):
    predictors = df.drop(target, axis=1)
    if t == 0:
        f = target + '~' + "+".join(predictors.columns)
        model_fit_results = ols(formula=f, data=boston_features).fit()
        print(model_fit_results.summary())
        return model_fit_results
    else:
        model_fit_results = LinearRegression().fit(predictors, df[target])
        print("sklearn Regression Results ")
        print("==============================================================================")  
        print("Dep. Variable:\t\t{}".format(target))      
        print("Model:\t\t\t{}".format(model_fit_results))
        print("==============================================================================")
        print("Intercept:\t\t{}".format(model_fit_results.intercept_))
        for i in range(0, len(predictors.columns.tolist())):
            print("{}:\t\t\t{}".format(predictors.columns[i], model_fit_results.coef_[i]))
        print("==============================================================================")
        return model_fit_results

target = 'MEDV'

# we need the target
boston_target = pd.DataFrame(data=boston.target, columns=[target])
#print(boston_target, "\n")
boston_features = pd.concat([boston_features, boston_target], axis=1, join='inner')
#print(boston_features.head(), "\n")

pred = boston_features.columns.tolist()
del pred[-1] # remove 'MEDV' since it is the target

mfr_ols = lin_reg_summary(boston_features, target, pred, t=0)

OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     144.9
Date:                Mon, 28 Oct 2019   Prob (F-statistic):          5.08e-153
Time:                        16:52:21   Log-Likelihood:                -1458.2
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     493   BIC:                             2997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       8.6442      3.189      2.711      0.007       2.379      14.91

## Run the same model in Scikit-learn

In [5]:
# Your code here - Check that the coefficients and intercept are the same as those from Statsmodels
mfr_skl = lin_reg_summary(boston_features, target, pred, t=1)

sklearn Regression Results 
Dep. Variable:		MEDV
Model:			LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Intercept:		8.644156137983725
CRIM:			-1.953802330561805
INDUS:			-0.8045754861251412
CHAS:			2.595867762517814
RM:			2.6465711106317364
AGE:			0.07939726661293223
DIS:			-10.096184652632433
PTRATIO:			-1.486665988772452
B:			3.841213903133081
LSTAT:			-5.628793689508368
RAD_6_24:			1.3379631728800436
TAX_270_360:			-1.2597761198832762
TAX_360_712:			-2.146061878770741


## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

In [6]:
# define min_max and standardize functions to make this easier/cleaner
def min_max(df, feature, feature_instance, transform = None):
    t_feat_inst = feature_instance if transform is None else transform(feature_instance)
    t_max_df_feat = max(df[feature] if transform is None else transform(df[feature]))
    t_min_df_feat = min(df[feature] if transform is None else transform(df[feature]))
    return (t_feat_inst - t_min_df_feat)/(t_max_df_feat - t_min_df_feat)

def standardize(df, feature, feature_instance, transform = None):
    t_feat_inst = feature_instance if transform is None else transform(feature_instance)
    t_mean_df_feat = np.mean(df[feature] if transform is None else transform(df[feature]))
    t_sqrt_var_df_feat = np.sqrt(np.var(df[feature] if transform is None else transform(df[feature])))
    return (t_feat_inst - t_mean_df_feat)/t_sqrt_var_df_feat

In [7]:
# assumes inputs have NOT been scaled or transformed - this function does that
def predict_price(mfr, CRIM, INDUS, CHAS, RM, AGE, DIS, PTRATIO, B, LSTAT, RAD, TAX):
    bf2 = pd.DataFrame(boston.data, columns = boston.feature_names)
    # We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
    minmax_B = min_max(bf2, 'B', B)
    minmax_CRIM = min_max(bf2, 'CRIM', CRIM, np.log)
    minmax_DIS = min_max(bf2, 'DIS', DIS, np.log)
    # We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE")
    std_AGE = standardize(bf2, 'AGE', AGE)
    std_INDUS = standardize(bf2, 'INDUS', INDUS, np.log)
    std_LSTAT = standardize(bf2, 'LSTAT', LSTAT, np.log)
    std_PTRATIO = standardize(bf2, 'PTRATIO', PTRATIO, np.log)
    bins_rad_2 = pd.cut([RAD], [0, 6, 24])
    rad_dummy_2 = pd.get_dummies(bins_rad_2, prefix="RAD", drop_first=True).rename(columns=lambda col: col.replace(" ", "").replace(",", "_").replace("(", "").replace("]", ""))
    bins_tax_2 = pd.cut([TAX], [0, 270, 360, 712])
    tax_dummy_2 = pd.get_dummies(bins_tax_2, prefix="TAX", drop_first=True).rename(columns=lambda col: col.replace(" ", "").replace(",", "_").replace("(", "").replace("]", ""))
    pred_basis = pd.concat([pd.DataFrame([[minmax_CRIM, std_INDUS, CHAS, RM, std_AGE, minmax_DIS, std_PTRATIO, minmax_B, std_LSTAT]], columns=['CRIM', 'INDUS', 'CHAS', 'RM','AGE', 'DIS', 'PTRATIO', 'B', 'LSTAT']), rad_dummy_2, tax_dummy_2], axis=1, join='inner')
    pred_basis
    return mfr.predict(pred_basis) # MEDV: median value of owner-occupied homes in $1000s.

In [8]:
CRIM=0.15
INDUS=6.07
CHAS=1
RM=6.1
AGE=33.2
DIS=7.6
PTRATIO=17
B=383
LSTAT=10.87
RAD=8
TAX=284
pred_medv = predict_price(mfr_ols, CRIM, INDUS, CHAS, RM, AGE, DIS, PTRATIO, B, LSTAT, RAD, TAX)
print("With values CRIM={}, CHAS={}, RM={}, AGE={}, DIS={}, PTRAIO={}, B={}, LSTAT={}, RAD={}, and TAX={}, the model predicts the value of the home to be: ${}".format(INDUS, CHAS, RM, AGE, DIS, PTRATIO, B, LSTAT, RAD, TAX, round(pred_medv[0]*1000, 2)))

With values CRIM=6.07, CHAS=1, RM=6.1, AGE=33.2, DIS=7.6, PTRAIO=17, B=383, LSTAT=10.87, RAD=8, and TAX=284, the model predicts the value of the home to be: $23431.98


## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.